In [4]:
!pip3 install scrapetube

In [5]:
import pandas as pd
import time
import scrapetube


In [6]:
def extract_video_data(videos):
    """
    Extracts relevant information from a list of video dictionaries and returns a Pandas DataFrame.
    """
    data = []
    
    for video in videos:
        video_id = video.get('videoId', None)
        title = video.get('title', {}).get('runs', [{}])[0].get('text', None)
        channel_name = video.get('longBylineText', {}).get('runs', [{}])[0].get('text', None)
        channel_id = video.get('longBylineText', {}).get('runs', [{}])[0].get('navigationEndpoint', {}).get('browseEndpoint', {}).get('browseId', None)
        published_time = video.get('publishedTimeText', {}).get('simpleText', None)
        duration = video.get('lengthText', {}).get('simpleText', None)
        views = video.get('viewCountText', {}).get('simpleText', None)
        video_url = f"https://www.youtube.com/watch?v={video_id}" if video_id else None
        channel_url = f"https://www.youtube.com/channel/{channel_id}" if channel_id else None
        
        # Extract thumbnails (choosing highest resolution available)
        thumbnails = video.get('thumbnail', {}).get('thumbnails', [])
        thumbnail_url = thumbnails[-1]['url'] if thumbnails else None
        
        data.append({
            "video_id": video_id,
            "title": title,
            "channel_name": channel_name,
            "channel_id": channel_id,
            "published_time": published_time,
            "duration": duration,
            "views": views,
            "video_url": video_url,
            "channel_url": channel_url,
            "thumbnail_url": thumbnail_url,
        })
    
    return pd.DataFrame(data)

In [3]:
hashtags = [
    "ChildbirthJourney", "BirthStory", "LaborAndDelivery", "MomLife", "BirthVlog",
    "PregnancyJourney", "NaturalBirth", "HospitalBirth", "WaterBirth", "HomeBirth",
    "BirthExperience", "LaborPain", "BirthingProcess", "DoulaSupport", "MidwifeCare",
    "BirthPrep", "PregnancyToBaby", "BabyIsComing", "BirthWithoutFear",
    "PostpartumJourney", "PregnancyToParenthood", "BirthEducation", "BabyOnTheWay",
    "BirthSupport", "EmpoweredBirth", "Hypnobirthing", "PositiveBirth",
    "PregnancyLife", "LaborAndLove", "BirthOfABaby", "BabyDelivery", "BirthWarrior",
    "RainbowBaby", "FirstTimeMom", "PregnantAndProud", "PregnancyGoals", "BirthTips",
    "LaborAndBeyond", "MiracleOfLife", "BirthIsBeautiful", "PregnancyMilestones",
    "WelcomeBaby", "BirthPlan", "PregnancyGlow", "LaborAndSupport", "BirthMagic",
    "MomToBeJourney", "NewbornArrival", "BirthAndBonding", "PregnancyAndBirth",
    "Birth", "Childbirth", "BirthTok", "ChildbirthEducation", "ChildbirthTips",
    "Pregnant", "Pregnancy", "GivingBirth", "Labor", "PregnantTikTok",
    "LabourPain", "PregnantTikTokers", "EpiduralBirth", "ChildbirthClass",
    "NaturalChildbirth", "BirthTipsForMoms", "CSection", "CSections",
    "CesarianSection"
]

In [28]:
test_hashtags = [
    "ChildbirthJourney", "BirthStory"
]

In [7]:
import datetime 
from tqdm import tqdm
# Main function to scrape videos
def scrape_youtube_videos(hashtags, limit_per_hashtag=30000):
    all_videos = []

    for hashtag in tqdm(hashtags, desc="Scraping Hashtags", unit="hashtag"):
        print(f"Scraping videos for hashtag: {hashtag}  | {datetime.datetime.now()+datetime.timedelta(hours=2)}")
        
        videos = list(scrapetube.get_search(query=f"{hashtag}", limit=limit_per_hashtag, sort_by="relevance", results_type="video", sleep=3))
        
        # Extract relevant data
        df = extract_video_data(videos)
        df["hashtag"] = hashtag  # Add hashtag column for reference
        # print(df)
        print(f"Total length: {len(df)}")
        
        all_videos.append(df)
        
        # Sleep to avoid getting blocked by YouTube
        time.sleep(10)
    
    # Combine all data
    final_df = pd.concat(all_videos, ignore_index=True)
    
    # Save to CSV
    final_df.to_csv("youtube_childbirth_videos_extra_tags.csv", index=False)
    print("Data saved to youtube_childbirth_videos_extra_tags.csv")

In [ ]:
scrape_youtube_videos(hashtags,limit_per_hashtag=30000)

In [37]:
res_df = pd.read_csv("/kaggle/working/youtube_childbirth_videos.csv")
res_df

,video_id,title,channel_name,channel_id,published_time,duration,views,video_url,channel_url,thumbnail_url,hashtag
0,6W0OwgtAnP4,Mums Share Their Childbirth Journey - EP6 | Co...,SAYS,UCI6hKuCfwBkd0BEPsPph3Lw,1 year ago,6:45,"60,530 views",https://www.youtube.com/watch?v=6W0OwgtAnP4,https://www.youtube.com/channel/UCI6hKuCfwBkd0...,https://i.ytimg.com/vi/6W0OwgtAnP4/hq720.jpg?s...,ChildbirthJourney
1,ji98SYdIPoY,BIRTH VLOG! Labour & Delivery Of Our Baby Boy ...,Taylor R,UCuySkUHjdgugMw61waqFsRg,2 years ago,15:32,"5,635,620 views",https://www.youtube.com/watch?v=ji98SYdIPoY,https://www.youtube.com/channel/UCuySkUHjdgugM...,https://i.ytimg.com/vi/ji98SYdIPoY/hq720.jpg?s...,ChildbirthJourney
2,bA80Y7aNUwc,Joanna's High-Risk Childbirth Journey,Medical City Las Colinas,UCkENu3nITMxcNNJ5pZKmcxQ,1 year ago,2:44,284 views,https://www.youtube.com/watch?v=bA80Y7aNUwc,https://www.youtube.com/channel/UCkENu3nITMxcN...,https://i.ytimg.com/vi/bA80Y7aNUwc/hq720.jpg?s...,ChildbirthJourney
3,6V77p20u69U,Sextuplets: Our Multiple Birth Medical Journey...,Real Stories,UCu4XcDBdnZkV6-5z2f16M0g,7 years ago,46:50,"1,449,041 views",https://www.youtube.com/watch?v=6V77p20u69U,https://www.youtube.com/channel/UCu4XcDBdnZkV6...,https://i.ytimg.com/vi/6V77p20u69U/hq720.jpg?s...,ChildbirthJourney
4,xm5RqzzwnKI,VHC OB Journey: Going Home after Childbirth - ...,VHC Health,UCB3EftUTf_b0moo9cg6gR0g,3 years ago,7:28,"5,001 views",https://www.youtube.com/watch?v=xm5RqzzwnKI,https://www.youtube.com/channel/UCB3EftUTf_b0m...,https://i.ytimg.com/vi/xm5RqzzwnKI/hqdefault.j...,ChildbirthJourney
...,...,...,...,...,...,...,...,...,...,...,...
33753,w4l-1DCM3vs,ATAL Sponsored 6 Day Online FDP Day 05 Session 10,Gnan cse Hub,UCAaz79jK9xu7WGDQUuLMc-A,11 hours ago,1:11:14,3 views,https://www.youtube.com/watch?v=w4l-1DCM3vs,https://www.youtube.com/channel/UCAaz79jK9xu7W...,https://i.ytimg.com/vi/w4l-1DCM3vs/hq720.jpg?s...,CesarianSection
33754,gM1itub5-RE,NEET PG MCQ CHALLENGE OBSTETRICS AND GYNAECOLO...,NEET Academy,UCDRq7Pd-0D6ZctO2ATO39NQ,Streamed 2 days ago,11:27,No views,https://www.youtube.com/watch?v=gM1itub5-RE,https://www.youtube.com/channel/UCDRq7Pd-0D6Zc...,https://i.ytimg.com/vi/gM1itub5-RE/hq720.jpg?s...,CesarianSection
33755,Kdn0H3FfZlo,RRB staff nurse officer exam 2025 | RRB staff ...,Test mcqs Master,UCFcvaH4lwOoSWwaXN_XvxtQ,7 days ago,9:53,"3,883 views",https://www.youtube.com/watch?v=Kdn0H3FfZlo,https://www.youtube.com/channel/UCFcvaH4lwOoSW...,https://i.ytimg.com/vi/Kdn0H3FfZlo/hq720.jpg?s...,CesarianSection
33756,MPitelE-uAY,WEF 2025: Investing in Women's Health,CNBC Africa,UCsba91UGiQLFOb5DN3Z_AdQ,9 days ago,45:26,283 views,https://www.youtube.com/watch?v=MPitelE-uAY,https://www.youtube.com/channel/UCsba91UGiQLFO...,https://i.ytimg.com/vi/MPitelE-uAY/hq720.jpg?s...,CesarianSection


In [4]:
query_variations = [
    # Existing queries
    "CSection", "C Section", "C-Section", "Cesarean Section", "Cesarean Birth",
    "CSection birth", "CSection recovery", "Childbirth", "Giving Birth", 
    "Labor and Delivery", "Birth Process", "Natural Birth", "Hospital Birth",
    "Pregnancy", "Pregnant Mom", "Pregnancy Journey", "Pregnancy Tips", 
    "Pregnancy to Parenthood", "Pregnancy Milestones",

    # Additional childbirth-related searches
    "Home Birth", "Home Baby Delivery", "Water Birth at Home", "Midwife Birth",
    "Childbirth Myths", "C Section Myths", "Pain Management During Labor",
    "Hypnobirthing Techniques", "Unmedicated Birth", "Vaginal Birth After C Section",
    "VBAC Birth Stories", "Emergency C Section", "Planned C Section",
    "C Section Recovery Tips", "Giving Birth Naturally", "Fast Labor",
    "Long Labor Stories", "Labor Induction", "Labor and Epidural",
    "Labor without Epidural", "Birth Complications", "Breech Birth",
    "Twin Birth Stories", "Surprise Baby Birth", "Unexpected Home Birth",
    "Birth Center vs Hospital Birth", "Doula vs Midwife", "Birth Partner Support",
    "How to Prepare for Labor", "Signs of Labor", "Real Labor vs Braxton Hicks",
    "What Labor Feels Like", "What Happens During Labor", "Stages of Labor",
    "Perineal Tearing Prevention", "Epidural Pros and Cons", "Birth Photography",
    "Childbirth Education Classes", "Birth Stories from First-Time Moms",
    "Birthing Positions", "Water Breaking Signs", "Signs You Are in Labor",
    "How to Push During Birth", "When to Go to the Hospital for Labor",
    "Natural Birth Pain Management", "Fastest Ways to Go Into Labor",
    "Early Labor Signs", "Third Trimester Labor Signs", "What to Expect During Labor",
    "Hospital Bag Checklist for Labor", "Labor and Delivery Nurse Advice",
    "Coping with Labor Pain", "Postpartum Recovery", "First 24 Hours After Birth",
    "Breastfeeding After Birth", "Postpartum Depression", "Postpartum Healing",
    "Newborn Care After Birth", "Bringing Baby Home", "Fourth Trimester",
    "Baby’s First Moments After Birth", "Delayed Cord Clamping",
    "Newborn Skin to Skin Contact", "First-Time Dad in Delivery Room",
    "Newborn Sleep in First Week", "How to Hold a Newborn", "Postpartum Belly Recovery",
    "Pelvic Floor Exercises After Birth", "Postpartum Bleeding", "Postpartum Self-Care",
    "How to Swaddle a Baby", "Caring for Baby’s Umbilical Cord", 
    "Postpartum Anxiety", "Mom’s First 6 Weeks After Birth", "When Can I Exercise After Birth?",
    "How to Recover from Birth Faster", "Newborn Jaundice", "Postpartum Hair Loss",
    "C-Section Scar Care", "How to Bond with Newborn", "Best Labor and Delivery Tips"
]


In [7]:
scrape_youtube_videos(query_variations,limit_per_hashtag=40000)

Scraping videos for hashtag: CSection
Scraping videos for hashtag: C Section
Scraping videos for hashtag: C-Section
Scraping videos for hashtag: Cesarean Section
Scraping videos for hashtag: Cesarean Birth
Scraping videos for hashtag: CSection birth
Scraping videos for hashtag: CSection recovery
Scraping videos for hashtag: Childbirth
Scraping videos for hashtag: Giving Birth
Scraping videos for hashtag: Labor and Delivery
Scraping videos for hashtag: Birth Process
Scraping videos for hashtag: Natural Birth
Scraping videos for hashtag: Hospital Birth
Scraping videos for hashtag: Pregnancy
Scraping videos for hashtag: Pregnant Mom
Scraping videos for hashtag: Pregnancy Journey
Scraping videos for hashtag: Pregnancy Tips
Scraping videos for hashtag: Pregnancy to Parenthood
Scraping videos for hashtag: Pregnancy Milestones
Scraping videos for hashtag: Home Birth
Scraping videos for hashtag: Home Baby Delivery
Scraping videos for hashtag: Water Birth at Home
Scraping videos for hashtag: Mi

In [8]:
res_df = pd.read_csv("/kaggle/working/youtube_childbirth_videos_new_tags.csv")
res_df

,video_id,title,channel_name,channel_id,published_time,duration,views,video_url,channel_url,thumbnail_url,hashtag
0,hC_ZZssKcNs,Layers of C-Section Delivery ✨FASCINATING✨ #sh...,Pregnancy and Postpartum TV,UCIakpj3Xn5NBfRH9mfoG43w,1 year ago,0:28,"9,510,299 views",https://www.youtube.com/watch?v=hC_ZZssKcNs,https://www.youtube.com/channel/UCIakpj3Xn5NBf...,https://i.ytimg.com/vi/hC_ZZssKcNs/hq720_2.jpg...,CSection
1,J56tUGsJaSY,Don’t ❌ Let Your Nurse Do THIS After C-Section...,Pregnancy and Postpartum TV,UCIakpj3Xn5NBfRH9mfoG43w,1 year ago,0:19,"8,095,695 views",https://www.youtube.com/watch?v=J56tUGsJaSY,https://www.youtube.com/channel/UCIakpj3Xn5NBf...,https://i.ytimg.com/vi/J56tUGsJaSY/hq720.jpg?s...,CSection
2,VkxwN8xQz80,Primary vertex caesarean section (C-section) |...,TVASurg,UCIZ8lKJi8zKWkLsBRGSyLbw,6 years ago,3:44,"17,131,065 views",https://www.youtube.com/watch?v=VkxwN8xQz80,https://www.youtube.com/channel/UCIZ8lKJi8zKWk...,https://i.ytimg.com/vi/VkxwN8xQz80/hq720.jpg?s...,CSection
3,IWPrUQg8AeM,CESAREAN SECTION DELIVERY,THE WHITE ARMY,UCldeP-YJjm_jWgCf96_Pc_A,3 years ago,0:30,"1,430,584 views",https://www.youtube.com/watch?v=IWPrUQg8AeM,https://www.youtube.com/channel/UCldeP-YJjm_jW...,https://i.ytimg.com/vi/IWPrUQg8AeM/hq720_2.jpg...,CSection
4,JSDOGpZ9Dx4,Newborn Gets Gash on Face From C-Section Scalpel,Inside Edition,UC9k-yiEpRHMNVOnOi_aQK8w,3 years ago,1:22,"2,656,926 views",https://www.youtube.com/watch?v=JSDOGpZ9Dx4,https://www.youtube.com/channel/UC9k-yiEpRHMNV...,https://i.ytimg.com/vi/JSDOGpZ9Dx4/hq720.jpg?s...,CSection
...,...,...,...,...,...,...,...,...,...,...,...
57943,oeA302ds6jo,The day Sona answered all the pregnancy rumour...,Sonakshi Sinha,UCCmCg9T6JDiv_yX5BUARnrQ,12 days ago,8:08,"22,838 views",https://www.youtube.com/watch?v=oeA302ds6jo,https://www.youtube.com/channel/UCCmCg9T6JDiv_...,https://i9.ytimg.com/vi/oeA302ds6jo/hq720_cust...,Best Labor and Delivery Tips
57944,gKlDTCXgXJ0,Natural Childbirth At Home || Emotional Baby B...,🌀Birth&Earth🌏,UCTm-bpeljg5yk_1YhNYPQWQ,4 days ago,3:23,546 views,https://www.youtube.com/watch?v=gKlDTCXgXJ0,https://www.youtube.com/channel/UCTm-bpeljg5yk...,https://i.ytimg.com/vi/gKlDTCXgXJ0/hq720.jpg?s...,Best Labor and Delivery Tips
57945,9nuALX5PABs,😨 ₹1 Lakh - 1 Month? Swiggy and Zomato | CITIZ...,Madan Gowri,UCY6KjrDBN_tIRFT_QNqQbRQ,6 days ago,42:25,"322,775 views",https://www.youtube.com/watch?v=9nuALX5PABs,https://www.youtube.com/channel/UCY6KjrDBN_tIR...,https://i9.ytimg.com/vi/9nuALX5PABs/hq720_cust...,Best Labor and Delivery Tips
57946,lb88_Keac6o,বেবি নরমাল ডেলিভারির পজিশনে আসলে কোন জায়গায় ...,Desi Queen ponchu👑,UCHyqldauxbng3rc3VWJ83Eg,3 days ago,8:34,"1,817 views",https://www.youtube.com/watch?v=lb88_Keac6o,https://www.youtube.com/channel/UCHyqldauxbng3...,https://i.ytimg.com/vi/lb88_Keac6o/hq720.jpg?s...,Best Labor and Delivery Tips


In [5]:
query_variations_extra = [
    # Labor & Delivery Experiences
    "What to know about epidural", "Pros and cons of epidural", "Epidural vs natural birth",
    "Epidural vs spinal block", "Vacuum assisted delivery", "Forceps delivery explained",
    "Triplet birth experience", "What does birth feel like", "Birth in an ambulance",
    "Baby born in car", "Giving birth alone", "Birth complications stories",
    "What happens if baby gets stuck during labor", "Giving birth with no doctor",
    "What to do if baby comes too fast", "What happens if labor stalls",
    "What is a precipitous labor", "Signs of fetal distress during labor",
    "When is labor considered high risk", "Emergency childbirth situations",

    # C-Section & VBAC Topics
    "Emergency C Section vs planned C Section", "C Section vs vaginal birth recovery",
    "What happens during a C Section", "What to expect after a C Section",
    "C Section birth stories", "Do you feel pressure during a C Section",
    "Why do some women need a C Section", "C Section vs natural birth pros and cons",
    "Best positions for VBAC", "VBAC success stories", "VBAC after multiple C Sections",
    "Why do some babies need forceps or vacuum", "Can you refuse a C Section",
    "Is VBAC safe", "What happens if a VBAC fails",

    # Rare & Unique Birth Scenarios
    "Breech birth without C Section", "Footling breech birth", "Cord prolapse birth",
    "Baby born en caul", "Can you give birth standing up", "What happens if you give birth at home unexpectedly",
    "Twins with different birthdays", "Identical vs fraternal twin birth stories",
    "How to prepare for twins birth", "Triplets natural birth vs C Section",
    "Quadruplets birth experience", "What happens during a silent birth",
    "What is a lotus birth", "Delayed cord clamping benefits",
    "Can you give birth in the ocean", "Unusual birth locations", "What is orgasmic birth",
    "Can labor be painless", "Do all women scream in labor",

    # Medical Interventions & Risks
    "Inducing labor at 37 weeks", "Is induction safe", "How long can you go overdue",
    "Membrane sweep pros and cons", "What happens if baby is overdue", "Risks of pitocin",
    "Amniotic fluid levels during labor", "Can water break without contractions",
    "What happens if water breaks too early", "Placenta previa birth stories",
    "Umbilical cord wrapped around neck birth stories", "What is shoulder dystocia",
    "Why does fetal heart rate drop during labor", "Baby’s first breath complications",
    "Neonatal resuscitation explained", "What happens if baby is born not breathing",
    "What is meconium aspiration syndrome", "What to do if baby is in distress",

    # Pain Management & Alternative Birth Methods
    "How does laughing gas work during labor", "Alternative pain relief for labor",
    "Hypnobirthing vs epidural", "Breathing techniques for labor", "Music therapy during labor",
    "Can meditation help with labor pain", "What is a TENS unit for labor", 
    "Does acupuncture help with labor", "Rebozo technique for labor",
    "How does water birth reduce pain", "Does eating dates help with labor",
    "Spinning babies techniques", "Why do some women squat during birth",
    "Perineal massage before labor", "How to prevent tearing during birth",
    
    # Postpartum & Newborn Care
    "How long does postpartum bleeding last", "What to do after giving birth",
    "How to care for perineal stitches", "Signs of postpartum hemorrhage",
    "What is postpartum eclampsia", "How to care for a newborn umbilical cord",
    "Newborn baby reflexes explained", "How to swaddle a newborn", 
    "How to get newborn to sleep", "How to help baby latch for breastfeeding",
    "Newborn breathing sounds", "Why do newborns sneeze so much",
    "How to do skin to skin with baby", "What is the golden hour after birth",
    "How long does it take for milk to come in after birth"
]


In [14]:
scrape_youtube_videos(query_variations_extra,limit_per_hashtag=40000)

Scraping videos for hashtag: What to know about epidural  | 2025-02-01 13:17:23.235808
Total length: 600
Scraping videos for hashtag: Pros and cons of epidural  | 2025-02-01 13:19:23.824184
Total length: 501
Scraping videos for hashtag: Epidural vs natural birth  | 2025-02-01 13:21:05.576899
Total length: 570
Scraping videos for hashtag: Epidural vs spinal block  | 2025-02-01 13:22:58.497655
Total length: 514
Scraping videos for hashtag: Vacuum assisted delivery  | 2025-02-01 13:24:40.293546
Total length: 491
Scraping videos for hashtag: Forceps delivery explained  | 2025-02-01 13:26:18.076339
Total length: 517
Scraping videos for hashtag: Triplet birth experience  | 2025-02-01 13:28:02.509486
Total length: 584
Scraping videos for hashtag: What does birth feel like  | 2025-02-01 13:29:58.728840
Total length: 575
Scraping videos for hashtag: Birth in an ambulance  | 2025-02-01 13:31:57.972888
Total length: 589
Scraping videos for hashtag: Baby born in car  | 2025-02-01 13:33:52.524841
T

In [15]:
res_df = pd.read_csv("/kaggle/working/youtube_childbirth_videos_extra_tags.csv")
len(res_df)

56456

In [6]:
search_queries = [
    # Medications & Pain Management
    "Safe medications during pregnancy", "What painkillers can I take during labor",
    "Epidural vs spinal anesthesia", "Side effects of epidural during labor",
    "Pain relief options for childbirth", "Opioids for labor pain",
    "Is fentanyl used during labor", "Nitrous oxide for pain relief in labor",
    "Morphine use during pregnancy", "Epidural vs pudendal block",
    "Natural alternatives to epidural", "Is laughing gas safe for labor",
    "How does a walking epidural work", "Risks of general anesthesia in C-Section",
    "Best birth positions for pain relief", "TENS machine for labor pain",
    "Does acupuncture help with labor pain", "Best essential oils for childbirth",
    "What is a pudendal nerve block", "When is IV pain medication used in labor",
    "Can epidurals cause back pain long-term", "Medication for postpartum pain relief",
    "Are opioids safe for postpartum recovery", "Using CBD for labor pain",
    "Homeopathic remedies for labor pain", "Do epidurals affect baby alertness",

    # Medical Procedures & Risks
    "What is a cervical ripening balloon", "How does a Foley catheter induce labor",
    "What is a sweep and stretch in pregnancy", "Amniotomy for labor induction",
    "Forceps vs vacuum delivery risks", "When is a blood transfusion needed during birth",
    "Can you refuse a C-Section", "What is shoulder dystocia",
    "Signs of uterine rupture during labor", "When do doctors perform an episiotomy",
    "Emergency hysterectomy after childbirth", "Postpartum hemorrhage treatments",
    "Placental abruption risks", "What is postpartum eclampsia", "HELLP syndrome birth risks",
    "Maternal sepsis symptoms", "Gestational hypertension effects on labor",
    "What is polyhydramnios", "Can oligohydramnios affect delivery",
    "Cord prolapse emergency delivery", "Placental calcification risks",
    "Umbilical cord knots and birth risks", "Blood clot risks in pregnancy",
    "What is Rh incompatibility in pregnancy", "Does gestational diabetes require induction",

    # High-Risk Pregnancies
    "Giving birth with preeclampsia", "What is placenta previa",
    "Can you deliver vaginally with placenta previa", "Placenta accreta C-Section risks",
    "How does gestational diabetes affect labor", "Can you be induced with gestational diabetes",
    "Premature birth risks", "Cervical insufficiency and preterm labor",
    "Twin pregnancy labor risks", "Triplet birth delivery options",
    "What happens if the umbilical cord is wrapped around the baby’s neck",
    "When do babies need oxygen after birth", "Signs of fetal distress before labor",
    "What is neonatal hypoxia", "When do doctors perform an emergency C-Section",
    "Polyhydramnios vs oligohydramnios risks", "How to manage pregnancy with lupus",
    "Does PCOS affect childbirth", "Pregnancy with heart conditions",
    "How does epilepsy affect pregnancy", "Can asthma worsen in pregnancy",
    "Risks of high BMI pregnancy", "Can anemia cause complications in labor",
    "Pregnancy with an incompetent cervix", "What is a cerclage for pregnancy",

    # Induction & Labor
    "What to expect during labor induction", "How does Pitocin work",
    "Pitocin vs Cytotec for induction", "What happens if induction fails",
    "Membrane stripping pros and cons", "Does eating spicy food really induce labor",
    "Inducing labor naturally at 39 weeks", "What is balloon induction for labor",
    "How long does labor last after induction", "Best ways to speed up labor",
    "Is nipple stimulation safe for labor induction", "Midwife's brew for labor",
    "What is the bishop score for labor induction", "Can castor oil induce labor safely",
    "Risks of overdue pregnancy", "Does walking help start labor",
    "Do contractions start suddenly", "What does labor pain feel like",
    "How long after mucus plug do you go into labor", "What is prodromal labor",
    "How many cm dilated before hospital admission", "How to soften the cervix naturally",

    # Hospital Birth & Care
    "Hospital birth vs birth center", "Do hospitals allow delayed cord clamping",
    "Can I refuse an IV during labor", "Do hospitals allow birthing tubs",
    "When is a C-Section medically necessary", "What is a birth plan checklist",
    "How to advocate for yourself during labor", "Can you refuse cervical checks during labor",
    "What happens during an emergency delivery", "How to prepare for hospital birth",
    "Hospital discharge rules after birth", "Do hospitals allow doulas in delivery rooms",
    "Can partners cut the umbilical cord", "Neonatal care for preemies",
    "What is kangaroo care for newborns", "What is a labor and delivery nurse",
    "Hospital visitor policies during birth", "How do hospitals handle stillbirths",
    "Why are newborns given vitamin K shots", "Why do hospitals give newborn eye ointment",
    "Why do some hospitals ban water births", "How long do hospitals monitor newborns",
    "When can newborns go home from NICU", "How to request a private hospital room",

    # Postpartum Recovery & Newborn Health
    "What is postpartum endometritis", "How to heal a perineal tear fast",
    "Best postpartum pain relievers", "What is postpartum thyroiditis",
    "Best ways to prevent postpartum depression", "Signs of infection after C-Section",
    "Postpartum uterine contractions", "What is diastasis recti after pregnancy",
    "Can you get pregnant while breastfeeding", "How long does postpartum anemia last",
    "When can I resume sex after childbirth", "Postpartum hair loss remedies",
    "Can you develop allergies after pregnancy", "How long should you rest after childbirth",
    "Postpartum insomnia solutions", "What is postpartum rage",
    "Can postpartum depression affect dads", "Postpartum psychosis symptoms",
    "Postpartum blood clot risks", "When does postpartum sweating stop",
    "How long does lochia last", "Postpartum thyroid dysfunction",
    "What is baby blues vs postpartum depression", "How long does it take for uterus to shrink",

    # Breastfeeding & Newborn Care
    "Breastfeeding vs formula feeding pros and cons", "How to increase breast milk supply",
    "Best lactation foods", "What is tongue-tie in newborns",
    "Signs of a good breastfeeding latch", "Why does my baby cluster feed",
    "Is it normal for newborns to lose weight after birth", "Newborn jaundice treatment",
    "How long do babies sleep in first weeks", "What are hunger cues in newborns",
    "What is colostrum and why is it important", "Can babies be allergic to breast milk",
    "How to relieve engorged breasts", "How long does it take for milk to come in after birth",
    "Can I breastfeed with mastitis", "How long should babies be swaddled",
    "How to burp a newborn correctly", "Can newborns sleep too much",
    "Best formula brands for newborns", "What is newborn reflex testing",
    "When do newborns start seeing clearly", "Why do babies grunt in sleep",
    "What is a tongue-tie release procedure",

    # Rare Birth Situations
    "Breech birth without C Section", "Cord prolapse emergency delivery",
    "Can you deliver twins naturally", "Triplet C Section experiences",
    "What is a dry birth", "Baby born in the amniotic sac",
    "What happens if the baby is born not breathing", "How long can a baby survive without oxygen",
    "How do hospitals handle stillbirths", "What is a cryptic pregnancy",
    "What is a phantom pregnancy", "What happens if labor stalls",
    "How do doctors resuscitate newborns", "What is micropreemie survival rate",
    "Why do some babies need oxygen after birth", "Signs of newborn breathing problems",
    "What happens if baby has meconium aspiration"
]

print(len(search_queries))


186


In [8]:
scrape_youtube_videos(search_queries,limit_per_hashtag=40000)

Scraping Hashtags:   0%|          | 0/186 [00:00<?, ?hashtag/s]

Scraping videos for hashtag: Safe medications during pregnancy  | 2025-02-04 18:09:35.864561
Total length: 592


Scraping Hashtags:   1%|          | 1/186 [02:01<6:15:16, 121.71s/hashtag]

Scraping videos for hashtag: What painkillers can I take during labor  | 2025-02-04 18:11:37.578515
Total length: 531


Scraping Hashtags:   1%|          | 2/186 [03:52<5:53:47, 115.37s/hashtag]

...

Scraping videos for hashtag: Signs of newborn breathing problems  | 2025-02-04 23:43:36.169046
Total length: 540


Scraping Hashtags:  99%|█████████▉| 185/186 [5:35:44<01:47, 107.62s/hashtag]

Scraping videos for hashtag: What happens if baby has meconium aspiration  | 2025-02-04 23:45:20.858743
Total length: 497


Scraping Hashtags: 100%|██████████| 186/186 [5:37:21<00:00, 108.83s/hashtag]


Data saved to youtube_childbirth_videos_extra_tags.csv


In [ ]:
df = pd.read_csv("/kaggle/working/youtube_childbirth_videos_extra_tags.csv")
len(df)

In [8]:
all_hashtags_used_in_project = []
all_hashtags_used_in_project.extend(hashtags)
all_hashtags_used_in_project.extend(query_variations)
all_hashtags_used_in_project.extend(query_variations_extra)
all_hashtags_used_in_project.extend(search_queries)

In [10]:
len(all_hashtags_used_in_project)

457